In [146]:
import os
import io
from PIL import Image, ImageDraw, ImageFont
import qrcode
from fpdf import FPDF
import tempfile
import pandas as pd

# Classe PDF customizada
class PDF(FPDF):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.add_page()

    def add_label(self, img_buffer, x, y, width, height):
        if self.page_no() == 0:
            self.add_page()
        with tempfile.NamedTemporaryFile(delete=False, suffix=".png") as temp:
            img_buffer.seek(0)
            temp.write(img_buffer.read())
            temp.flush()
        self.image(temp.name, x, y, width, height)
        os.unlink(temp.name)

# Função para salvar etiquetas como PDF
def save_labels_as_pdf(labels):
    with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
        pdf = PDF()
        pdf.set_auto_page_break(auto=True, margin=10)
        
        # Configurações da página A4 (210mm x 297mm)
        page_width = 210  # Largura da página em mm
        page_height = 297  # Altura da página em mm
        
        # Margens
        margin_horizontal =  6 # 90mm de margem lateral
        spacing_horizontal = 3  # 3mm de espaçamento horizontal
        margin_vertical = 0  # Nenhuma margem vertical
        
        # Número de etiquetas por linha e por coluna
        labels_per_row = 3  # Apenas duas etiquetas devido à margem lateral
        labels_per_column = 6  # Sem espaçamento vertical
        
        # Área útil para etiquetas após margens laterais
        available_width = page_width - 2 * margin_horizontal
        label_width = (available_width - spacing_horizontal) / labels_per_row
        label_height = page_height / labels_per_column  # Sem espaçamento vertical

        for index, label in enumerate(labels):
            if index % (labels_per_row * labels_per_column) == 0 and index != 0:
                pdf.add_page()  # Nova página após preencher a atual

            # Calcula a posição da etiqueta
            row = (index // labels_per_row) % labels_per_column
            col = index % labels_per_row
            
            x = margin_horizontal + col * (label_width + spacing_horizontal)
            y = row * label_height

            # Salva a etiqueta no PDF
            buffer = io.BytesIO()
            label.save(buffer, format='PNG')
            pdf.add_label(buffer, x, y, label_width, label_height)

        pdf.output(temp_file.name)
        return temp_file.name  # Retornar o caminho para o arquivo temporário

# Função para gerar QR code
def generate_qr_code(link):
    img = qrcode.make(link)
    buffer = io.BytesIO()
    img.save(buffer)
    buffer.seek(0)
    return buffer

# Função para dividir o nome a cada 3 palavras
def split_name_into_lines(name, words_per_line=3):
    words = name.split()
    lines = [" ".join(words[i:i + words_per_line]) for i in range(0, len(words), words_per_line)]
    return "\n".join(lines)

# Função para criar uma única etiqueta
def create_single_label(name, qr_code_link, config):
    label_width, label_height = 500, 450  # Ajuste para caber dentro do espaço disponível
    label = Image.new('RGB', (label_width, label_height), 'white')
    draw = ImageDraw.Draw(label)

    # Configuração da fonte
    try:
        font = ImageFont.truetype("arial.ttf", config['name_font_size'])
    except IOError:
        font = ImageFont.load_default()

    # Ajusta o nome com quebras de linha a cada 3 palavras
    formatted_name = split_name_into_lines(name, words_per_line=4)
    
    # Divide o nome em linhas
    name_lines = formatted_name.split('\n')
    line_height = config['name_font_size'] + 5  # Espaçamento entre as linhas

    # Calcula a posição Y inicial para o nome (centralizado verticalmente)
    total_text_height = len(name_lines) * line_height
    name_y = (label_height - total_text_height - config['qr_code_size'] - 20) // 2

    # Adiciona cada linha do nome
    for line in name_lines:
        name_width = draw.textlength(line, font=font)
        name_x = (label_width - name_width) / 2  # Centraliza horizontalmente
        draw.text((name_x, name_y), line, font=font, fill='black')
        name_y += line_height  # Move para a próxima linha
    
    # Centraliza o QR code na etiqueta
    qr_code_size = config['qr_code_size']
    qr_code_x = (label_width - qr_code_size) // 2  # Centraliza horizontalmente
    qr_code_y = name_y + 20  # Posição Y do QR code (abaixo do nome)

    # Adiciona QR Code
    qr_code_img = Image.open(generate_qr_code(qr_code_link)).resize((qr_code_size, qr_code_size))
    label.paste(qr_code_img, (int(qr_code_x), int(qr_code_y)))

    return label

# Função para criar etiquetas a partir de uma lista de dados
def create_labels_from_data(data, config):
    labels = []
    for item in data:
        name = item.get('TITLE', '')
        qr_code_link = item.get('URL', '')
        label_image = create_single_label(name, qr_code_link, config)
        labels.append(label_image)
    return labels

# Configuração de layout das etiquetas
config = {
    'name_y': 20,  # Posição Y inicial do nome
    'qr_code_size': 200,  # Tamanho do QR code ajustado
    'name_font_size': 17  # Tamanho da fonte do nome
}

# Carregar os dados do CSV
data = pd.read_csv("2025-02-27T12-30_export.csv")

# Converter o DataFrame para uma lista de dicionários
data = data[['TITLE', 'URL']].to_dict('records')  # Usar 'records' para obter uma lista de dicionários

# Gerar etiquetas
labels = create_labels_from_data(data, config)

# Salvar etiquetas como PDF
pdf_path = save_labels_as_pdf(labels)
print(f"PDF gerado e salvo em: {pdf_path}")


PDF gerado e salvo em: C:\Users\nonak\AppData\Local\Temp\tmpqpccniau.pdf
